In [24]:
import polars as pl
from datetime import timedelta
from functools import reduce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
import numpy as np

In [25]:
Xy_train = pl.read_parquet("/home/thomas/repos/simplify_deployment/data/potential_features/s1/Xy_train_s1.parquet")
Xy_train

datetime_utc,lag_25_si_cumulative_minute,lag_26_si_cumulative_minute,lag_27_si_cumulative_minute,lag_28_si_cumulative_minute,lag_70_loaD_ID_MW,lag_55_loaD_ID_MW,lag_40_loaD_ID_MW,lag_25_loaD_ID_MW,lag_10_loaD_ID_MW,lag_-5_loaD_ID_MW,lag_-20_loaD_ID_MW,lag_70_dsO_ID_MW,lag_55_dsO_ID_MW,lag_40_dsO_ID_MW,lag_25_dsO_ID_MW,lag_10_dsO_ID_MW,lag_-5_dsO_ID_MW,lag_-20_dsO_ID_MW,lag_70_xB_ID_MW,lag_55_xB_ID_MW,lag_40_xB_ID_MW,lag_25_xB_ID_MW,lag_10_xB_ID_MW,lag_-5_xB_ID_MW,lag_-20_xB_ID_MW,target
"datetime[μs, UTC]",f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32
2023-01-03 00:14:00 UTC,72.132004,84.448997,88.986,93.719002,8496.85,8409.88,8272.75,8158.64,7980.57,7805.43,7743.75,-4753.6,-4710.7,-4672.8,-4637.1,-4435.5,-4384.7,-4343.1,347.1,347.1,307.9,294.1,352.3,342.5,297.7,-261.561005
2023-01-03 00:29:00 UTC,-223.645996,-192.675003,-172.985001,-153.403,8409.88,8272.75,8158.64,7980.57,7805.43,7743.75,7693.4,-4710.7,-4672.8,-4637.1,-4435.5,-4384.7,-4343.1,-4302.8,347.1,307.9,294.1,352.3,342.5,297.7,264.0,-243.889999
2023-01-03 00:44:00 UTC,-265.730988,-269.908997,-277.419006,-285.411987,8272.75,8158.64,7980.57,7805.43,7743.75,7693.4,7546.94,-4672.8,-4637.1,-4435.5,-4384.7,-4343.1,-4302.8,-4149.1,307.9,294.1,352.3,342.5,297.7,264.0,879.3,-206.968994
2023-01-03 00:59:00 UTC,-211.822998,-219.957993,-222.531998,-236.660004,8158.64,7980.57,7805.43,7743.75,7693.4,7546.94,7475.51,-4637.1,-4435.5,-4384.7,-4343.1,-4302.8,-4149.1,-4145.5,294.1,352.3,342.5,297.7,264.0,879.3,803.1,-156.569
2023-01-03 01:14:00 UTC,-171.369003,-163.285004,-159.220001,-166.363998,7980.57,7805.43,7743.75,7693.4,7546.94,7475.51,7432.8,-4435.5,-4384.7,-4343.1,-4302.8,-4149.1,-4145.5,-4137.2,352.3,342.5,297.7,264.0,879.3,803.1,739.4,-215.707993
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-12-31 22:44:00 UTC,-80.912003,-82.282997,-79.370003,-84.052002,8876.66,8818.7,8935.97,8793.88,8693.87,8549.87,8463.37,-3529.5,-3544.3,-3466.7,-3448.1,-3418.8,-3382.7,-2733.6,-2394.0,-2476.8,-2016.3,-2088.6,-2134.1,-2267.0,-2404.8,11.205
2023-12-31 22:59:00 UTC,11.572,26.243,43.362999,48.563999,8818.7,8935.97,8793.88,8693.87,8549.87,8463.37,8348.92,-3544.3,-3466.7,-3448.1,-3418.8,-3382.7,-2733.6,-2695.9,-2476.8,-2016.3,-2088.6,-2134.1,-2267.0,-2404.8,-2497.4,10.864
2023-12-31 23:14:00 UTC,-1.215,-1.366,-6.663,-9.853,8935.97,8793.88,8693.87,8549.87,8463.37,8348.92,8237.88,-3466.7,-3448.1,-3418.8,-3382.7,-2733.6,-2695.9,-2645.8,-2016.3,-2088.6,-2134.1,-2267.0,-2404.8,-2497.4,-2602.0,0.0


In [26]:
Xy_test = pl.read_parquet("/home/thomas/repos/simplify_deployment/data/potential_features/s1/Xy_test_s1.parquet")
Xy_test

datetime_utc,lag_25_si_cumulative_minute,lag_26_si_cumulative_minute,lag_27_si_cumulative_minute,lag_28_si_cumulative_minute,lag_70_loaD_ID_MW,lag_55_loaD_ID_MW,lag_40_loaD_ID_MW,lag_25_loaD_ID_MW,lag_10_loaD_ID_MW,lag_-5_loaD_ID_MW,lag_-20_loaD_ID_MW,lag_70_dsO_ID_MW,lag_55_dsO_ID_MW,lag_40_dsO_ID_MW,lag_25_dsO_ID_MW,lag_10_dsO_ID_MW,lag_-5_dsO_ID_MW,lag_-20_dsO_ID_MW,lag_70_xB_ID_MW,lag_55_xB_ID_MW,lag_40_xB_ID_MW,lag_25_xB_ID_MW,lag_10_xB_ID_MW,lag_-5_xB_ID_MW,lag_-20_xB_ID_MW,target
"datetime[μs, UTC]",f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32
2024-01-03 00:14:00 UTC,166.626999,162.369995,155.291,152.613998,8848.64,8690.86,8579.12,8441.32,7851.67,7721.4,7641.56,-3279.5,-3228.1,-3180.1,-3130.2,-2842.2,-2776.7,-2730.4,-276.4,-332.8,-437.5,-516.6,-938.2,-999.1,-1085.4,-50.097
2024-01-03 00:29:00 UTC,-82.894997,-77.117996,-51.709,-37.901001,8690.86,8579.12,8441.32,7851.67,7721.4,7641.56,7541.0,-3228.1,-3180.1,-3130.2,-2842.2,-2776.7,-2730.4,-2678.8,-332.8,-437.5,-516.6,-938.2,-999.1,-1085.4,-1188.2,41.205002
2024-01-03 00:44:00 UTC,-38.172001,2.859,41.750999,53.191002,8579.12,8441.32,7851.67,7721.4,7641.56,7541.0,7702.71,-3180.1,-3130.2,-2842.2,-2776.7,-2730.4,-2678.8,-2527.6,-437.5,-516.6,-938.2,-999.1,-1085.4,-1188.2,-1404.6,10.788
2024-01-03 00:59:00 UTC,9.684,-20.532,-45.223,-59.348,8441.32,7851.67,7721.4,7641.56,7541.0,7702.71,7632.66,-3130.2,-2842.2,-2776.7,-2730.4,-2678.8,-2527.6,-2498.5,-516.6,-938.2,-999.1,-1085.4,-1188.2,-1404.6,-1384.4,28.538
2024-01-03 01:14:00 UTC,14.625,24.813,30.671,26.502001,7851.67,7721.4,7641.56,7541.0,7702.71,7632.66,7582.33,-2842.2,-2776.7,-2730.4,-2678.8,-2527.6,-2498.5,-2468.6,-938.2,-999.1,-1085.4,-1188.2,-1404.6,-1384.4,-1363.9,11.015
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-01-30 22:44:00 UTC,-186.979004,-192.843994,-162.369995,-201.731003,9857.66,9783.27,9710.15,9648.46,9541.15,9403.27,9339.07,-6876.1,-6860.0,-6785.0,-6743.5,-6691.8,-6639.6,-6313.0,3193.4,3032.1,3345.1,3333.6,3242.3,3090.5,3106.9,-140.723007
2024-01-30 22:59:00 UTC,-215.332993,-235.746002,-245.024994,-222.445999,9783.27,9710.15,9648.46,9541.15,9403.27,9339.07,9208.45,-6860.0,-6785.0,-6743.5,-6691.8,-6639.6,-6313.0,-6252.3,3032.1,3345.1,3333.6,3242.3,3090.5,3106.9,3046.7,-105.958
2024-01-30 23:14:00 UTC,-142.291,-133.606003,-142.483994,-177.619003,9710.15,9648.46,9541.15,9403.27,9339.07,9208.45,9083.75,-6785.0,-6743.5,-6691.8,-6639.6,-6313.0,-6252.3,-6198.4,3345.1,3333.6,3242.3,3090.5,3106.9,3046.7,2926.8,-7.133


In [27]:
model = LinearRegression()
model.fit(
    X = Xy_train.select(pl.exclude(["datetime_utc", "target"])),
    y = Xy_train.select(pl.col("target")).to_series()
)

LinearRegression()

In [28]:
prediction_raw = model.predict(
    Xy_test.select(pl.exclude(["datetime_utc", "target"]))
)
prediction_raw

array([   0.47777715,   17.82036688,  -19.8144817 , ..., -157.55647516,
        -90.59681673,    2.156189  ])

In [29]:
prediction_df = Xy_test.select(pl.col(["datetime_utc", "target"]))
prediction_df = prediction_df.with_columns(
    pl.Series("prediction", prediction_raw)
)
prediction_df

datetime_utc,target,prediction
"datetime[μs, UTC]",f32,f64
2024-01-03 00:14:00 UTC,-50.097,0.477777
2024-01-03 00:29:00 UTC,41.205002,17.820367
2024-01-03 00:44:00 UTC,10.788,-19.814482
2024-01-03 00:59:00 UTC,28.538,55.635779
2024-01-03 01:14:00 UTC,11.015,-53.673215
…,…,…
2024-01-30 22:44:00 UTC,-140.723007,-126.281796
2024-01-30 22:59:00 UTC,-105.958,-88.897434
2024-01-30 23:14:00 UTC,-7.133,-157.556475


In [30]:
root_mean_squared_error(
    y_true = prediction_df.select(pl.col("target")).to_series(),
    y_pred = prediction_df.select(pl.col("prediction")).to_series(),
)

112.98494392429994

In [31]:
prediction_df.write_parquet(
    "/home/thomas/repos/simplify_deployment/data/potential_features/s1/s1_predictions.parquet"
)

In [32]:
n = 10000
rmse_distribution = np.empty(shape = n)
for i in range(n):
    sample_df = prediction_df.sample(
        fraction=1,
        with_replacement=True
    )
    rmse_distribution[i] = root_mean_squared_error(
        y_true = sample_df.select(pl.col("target")).to_series(),
        y_pred = sample_df.select(pl.col("prediction")).to_series(),
    )

In [37]:
p80 = np.percentile(rmse_distribution, 80)
p80

114.69564329474053

In [38]:
p50 = np.percentile(rmse_distribution, 50)
p50

112.99122636572

In [39]:
p20 = np.percentile(rmse_distribution, 20)
p20

111.22497914137378

In [40]:
p50-p20

1.7662472243462162

In [41]:
p80-p50

1.7044169290205389